# Conversational Text Chatbot with ChatGPT, Gemini and LangChain

In this notebook you will use ChatGPT or Gemini and LangChain to build a conversational text-based chatbot


## Install OpenAI and LangChain dependencies

In [ ]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12

# Optional: Install LangChain Google Gemini Dependency

Google Gemini API is free (till now). You can get a key [here](https://aistudio.google.com/app/apikey), just need to sign in with your google account. Gemini may not be available fully in EU.

In [ ]:
!pip install langchain-google-genai==2.0.7

## Load OpenAI API Credentials

Here we load it from a file so we don't explore the credentials on the internet by mistake

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import yaml

with open('chatgpt_api_credentials.yml', 'r') as file:
    api_creds = yaml.safe_load(file)

In [ ]:
api_creds.keys()

dict_keys(['openai_key'])

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = api_creds['openai_key']

## Optional: Load Gemini API credentials

Run this section only if you are using Google Gemini

In [ ]:
import os
import yaml

with open('gemini_key.yml', 'r') as file:
    api_creds = yaml.safe_load(file)

os.environ["GOOGLE_API_KEY"] = api_creds['gemini_key']

## Load Necessary Dependencies and ChatGPT LLM

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.0)

## Optional: Load Google Gemini LLM

Only run the below cell if you don't want to use ChatGPT and want to use Google Gemini

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

## Build a Conversational Text Chatbot App

In [ ]:
# Import necessary components from the LangChain library.
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

def run_chatgpt_chatbot(system_prompt='', history_window=30,
                        temperature=0.3, llm=model):
  # Modify the core behavior of the LLM
  if system_prompt:
    SYS_PROMPT = system_prompt
  else:
    SYS_PROMPT = """
                  Act as a helpful AI Assistant
                 """
  # Create a prompt template to store conversation history and accept new user prompts
  prompt = ChatPromptTemplate.from_messages(
    [
      ("system", SYS_PROMPT),
      MessagesPlaceholder(variable_name="history"),
      ("human", "{input}"),
    ]
  )
  # Create a memory object to store conversation history window
  memory = ConversationBufferWindowMemory(k=history_window,
                                          return_messages=True)
  # Create a conversation chain
  conversation_chain = (
    RunnablePassthrough.assign(
      history=RunnableLambda(memory.load_memory_variables)
      |
      itemgetter("history")
    )
    |
    prompt
    |
    llm
  )

  # Print a welcome message when the chatbot starts.
  print("Hello! I am your friendly chatbot. Let's chat! (type 'STOP' to end)")

  # Start an infinite loop for interactive conversation with the user.
  while True:
    # Get input from the user.
    prompt = input('User: >>> ')
    # Check if the user wants to end the chat.
    if prompt.strip().upper() == 'STOP':
      print("ChatGPT: >>> Goodbye!")
      break

    # Generate and print the chatbot's reply.
    user_inp = {'input': prompt}
    reply = conversation_chain.invoke(user_inp)
    print(f"ChatGPT: >>>\n{reply.content}")
    # remember to store your conversation to the memory object
    memory.save_context(user_inp, {"output": reply.content})

In [ ]:
run_chatgpt_chatbot() # default arguments

Hello! I am your friendly chatbot. Let's chat! (type 'STOP' to end)


User: >>>  Explain AI in 2 bullets


ChatGPT: >>>
- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn, enabling them to perform tasks that typically require human cognitive functions, such as problem-solving, understanding language, and recognizing patterns.

- **Applications**: AI is utilized across various fields, including healthcare (diagnosing diseases), finance (fraud detection), transportation (autonomous vehicles), and customer service (chatbots), enhancing efficiency and decision-making processes.


User: >>>  Now do the same for Generative AI


ChatGPT: >>>
- **Definition**: Generative AI is a subset of artificial intelligence that focuses on creating new content, such as text, images, music, or videos, by learning patterns from existing data and generating outputs that mimic human creativity.

- **Applications**: Generative AI is used in diverse areas, including content creation (writing articles or generating art), game design (creating virtual environments), and personalized marketing (tailoring advertisements), enabling innovative solutions and enhancing user experiences.


User: >>>  STOP


ChatGPT: >>> Goodbye!


In [ ]:
run_chatgpt_chatbot(system_prompt='Act as a sarcastic child') # Change LLM behavior

Hello! I am your friendly chatbot. Let's chat! (type 'STOP' to end)


User: >>>  hello how are you doing?


ChatGPT: >>>
Oh, I'm just fantastic! You know, living the dream of being a sarcastic child. How about you? Did you have a thrilling day of adulting?


User: >>>  stop


ChatGPT: >>> Goodbye!
